In [15]:
import torch
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe
import spacy
import numpy as np

In [16]:
spacy_en = spacy.load('en')

def tokenizer(text): # create a tokenizer function
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [17]:
TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True, fix_length=150)
LABEL = data.Field(sequential=False, use_vocab=True)

In [18]:
pos = data.TabularDataset(
    path='./IMDB Dataset.csv', format='csv',
    fields=[('text', TEXT),
        ('labels', LABEL)])

In [19]:
TEXT.build_vocab(pos, vectors=GloVe(name='6B', dim=300))
LABEL.build_vocab(pos)

In [25]:
print(len(pos))

50001


In [21]:
train_iter = data.BucketIterator(pos,batch_size=32)

In [22]:
print(type(train_iter))

<class 'torchtext.data.iterator.BucketIterator'>


In [23]:
batch = next(iter(train_iter))
print(batch)
print("batch text: ", batch.text) # 对应 Fileld 的 name
print("batch label: ", batch.labels)


[torchtext.data.batch.Batch of size 32]
	[.text]:[torch.LongTensor of size 150x32]
	[.labels]:[torch.LongTensor of size 32]
batch text:  tensor([[   13,    12,    12,  ...,    12,    13, 13660],
        [    9,   128,   433,  ...,    89,    23,  1337],
        [   37,   950,     8,  ...,    28,  2112,  2122],
        ...,
        [    4,     2,   336,  ...,    42,     1,    10],
        [    2,   613,     7,  ...,    15,     1,    11],
        [  911,  5783,  7755,  ...,    58,     1,     2]])
batch label:  tensor([2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2,
        1, 1, 1, 2, 1, 1, 1, 2])


In [24]:
print(len(batch))

32
